# SW-2b-Python-RDFBasics

**Sidetrack Python** : [<< SW-2-CSharp-RDFBasics](SW-2-CSharp-RDFBasics.ipynb) | [Index](README.md) | [SW-3-CSharp-GraphOperations >>](SW-3-CSharp-GraphOperations.ipynb)

## RDF en Python avec rdflib - Fondamentaux

Ce notebook est un **sidetrack optionnel** qui presente l'equivalent Python des concepts du notebook SW-2 (dotNetRDF). Vous y decouvrirez **rdflib**, la bibliotheque de reference pour manipuler des graphes RDF en Python.

### Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. Creer et manipuler des graphes RDF avec rdflib
2. Utiliser les differents types de noeuds (URI, blank nodes, litteraux)
3. Serialiser des graphes dans differents formats (Turtle, N-Triples, JSON-LD)
4. Faire la correspondance entre dotNetRDF et rdflib

### Prerequis
- SW-2-CSharp-RDFBasics recommande (pour la comprehension conceptuelle)
- Python 3.10+

### Duree estimee : 30 minutes

> **Note** : Ce notebook se concentre sur les fondamentaux RDF. Pour SPARQL en Python, voir le sidetrack **SW-4b-Python-SPARQL**.

---

## 1. Installation et Imports

**rdflib** est le coeur de l'ecosysteme Python pour le Web Semantique. C'est l'equivalent direct de **dotNetRDF** pour .NET.

In [ ]:
# Installation silencieuse de rdflib
%pip install -q rdflib

### Interpretation

L'option `-q` (quiet) reduit la verbosite de pip. Une fois installe, rdflib offre une API pythonique pour toutes les operations RDF.

In [ ]:
from rdflib import Graph, URIRef, Literal, BNode, Namespace
from rdflib.namespace import RDF, RDFS, XSD, FOAF, DC

print("Imports rdflib prets.")
print(f"rdflib version : {Graph.__module__}")
print()
print("Namespaces standards W3C :")
print(f"  RDF  : {RDF}")
print(f"  RDFS : {RDFS}")
print(f"  XSD  : {XSD}")
print(f"  FOAF : {FOAF}")

### Interpretation

rdflib fournit des namespaces preconfigures pour les vocabulaires standards du W3C :
- `RDF` : types et proprietes de base (`rdf:type`, `rdf:Property`)
- `RDFS` : hierarchie de classes/proprietes (`rdfs:subClassOf`, `rdfs:label`)
- `XSD` : types de donnees XML Schema (`xsd:integer`, `xsd:dateTime`)
- `FOAF` : description de personnes (`foaf:name`, `foaf:knows`)

| Classe rdflib | Equivalent dotNetRDF |
|---------------|---------------------|
| `Graph` | `Graph` |
| `URIRef` | `IUriNode` |
| `BNode` | `IBlankNode` |
| `Literal` | `ILiteralNode` |
| `Namespace` | `UriFactory` / `PrefixMapper` |

---

## 2. Creer un Graphe et Ajouter des Triples

Un graphe RDF est un ensemble de triples `(sujet, predicat, objet)`. En rdflib, on utilise la methode `add()` avec un **tuple Python**.

In [ ]:
# Create a new empty graph
g = Graph()

# Define a custom namespace
EX = Namespace("http://example.org/")
g.bind("ex", EX)
g.bind("foaf", FOAF)

# Add triples about a person
g.add((EX.Alice, RDF.type, FOAF.Person))
g.add((EX.Alice, FOAF.name, Literal("Alice Dupont")))
g.add((EX.Alice, FOAF.age, Literal(30, datatype=XSD.integer)))
g.add((EX.Alice, FOAF.knows, EX.Bob))

g.add((EX.Bob, RDF.type, FOAF.Person))
g.add((EX.Bob, FOAF.name, Literal("Bob Martin")))
g.add((EX.Bob, FOAF.age, Literal(25, datatype=XSD.integer)))

print(f"Nombre de triples dans le graphe : {len(g)}")
print()

# Iterate over all triples
print("Triples dans le graphe :")
for s, p, o in g:
    print(f"  {s.n3(g.namespace_manager)} -- {p.n3(g.namespace_manager)} --> {o.n3(g.namespace_manager)}")

### Interpretation : Creation de graphe

**Sortie obtenue** : 7 triples decrivant deux personnes (Alice et Bob) et leur relation.

| Operation | Syntaxe rdflib | Syntaxe dotNetRDF |
|-----------|---------------|-------------------|
| Namespace personnalise | `EX = Namespace("http://example.org/")` | `UriFactory.Create("http://example.org/")` |
| Binding de prefixe | `g.bind("ex", EX)` | `g.NamespaceMap.AddNamespace("ex", uri)` |
| Ajouter un triple | `g.add((s, p, o))` | `g.Assert(new Triple(s, p, o))` |
| Compter les triples | `len(g)` | `g.Triples.Count` |
| Iterer les triples | `for s, p, o in g:` | `foreach (var t in g.Triples)` |

**Points cles** :
1. `g.add()` prend un **tuple Python** `(s, p, o)` - contrairement a dotNetRDF qui utilise un objet `Triple`
2. `EX.Alice` est un raccourci pour `URIRef("http://example.org/Alice")`
3. `g.bind()` associe un prefixe pour la serialisation (ex: `ex:Alice` au lieu de l'URI complete)
4. La methode `.n3(g.namespace_manager)` formate un noeud avec les prefixes (plus lisible)

---

## 3. Types de Noeuds

RDF definit trois types de noeuds. Voici comment rdflib les represente.

In [ ]:
# 1. URIRef - identifies a resource by its URI
uri_node = URIRef("http://example.org/Alice")
print(f"URIRef       : {uri_node}")
print(f"  type       : {type(uri_node).__name__}")
print()

# 2. BNode - anonymous node (no URI)
blank = BNode()
print(f"BNode        : {blank}")
print(f"  type       : {type(blank).__name__}")
print()

# 3. Literal - data values
lit_simple = Literal("Hello World")
lit_lang   = Literal("Bonjour le monde", lang="fr")
lit_int    = Literal(42, datatype=XSD.integer)
lit_float  = Literal(3.14, datatype=XSD.double)
lit_bool   = Literal(True, datatype=XSD.boolean)
lit_date   = Literal("2025-01-15", datatype=XSD.date)

print("Literal examples:")
for name, lit in [("simple", lit_simple), ("lang=fr", lit_lang),
                   ("integer", lit_int), ("double", lit_float),
                   ("boolean", lit_bool), ("date", lit_date)]:
    dt = lit.datatype if lit.datatype else "(none)"
    lg = lit.language if lit.language else "(none)"
    print(f"  {name:10s} : value={lit.toPython()!r:30s}  datatype={dt}  lang={lg}")

### Interpretation : Types de noeuds

| Type | Classe rdflib | Notation Turtle | Equivalent dotNetRDF |
|------|--------------|-----------------|---------------------|
| **URI** | `URIRef` | `<http://...>` ou `ex:Alice` | `g.CreateUriNode(uri)` |
| **Blank Node** | `BNode` | `_:b0` | `g.CreateBlankNode()` |
| **Litteral simple** | `Literal("texte")` | `"texte"` | `g.CreateLiteralNode("texte")` |
| **Litteral type** | `Literal(42, datatype=XSD.integer)` | `"42"^^xsd:integer` | `g.CreateLiteralNode("42", XSD.Integer)` |
| **Litteral avec langue** | `Literal("Bonjour", lang="fr")` | `"Bonjour"@fr` | `g.CreateLiteralNode("Bonjour", "fr")` |

**Points cles** :
1. La methode `.toPython()` convertit un litteral rdflib en type Python natif
2. Un litteral ne peut **pas** avoir a la fois un `datatype` et un `lang` (contrainte RDF)
3. Les `BNode` recoivent un identifiant unique auto-genere

---

## 4. Serialisation - Formats de Sortie

Un graphe RDF peut etre serialise dans plusieurs formats standards.

In [ ]:
# Serialize to Turtle (compact, human-readable)
turtle_output = g.serialize(format="turtle")
print("=== Format Turtle ===")
print(turtle_output)

### Interpretation : Format Turtle

Le format **Turtle** est le plus lisible pour les humains :
- Utilise des prefixes (`ex:`, `foaf:`)
- Regroupe les proprietes d'un meme sujet avec `;`
- Simplifie `rdf:type` en mot-cle `a`

Equivalent dotNetRDF : `CompressingTurtleWriter`

In [ ]:
# Serialize to N-Triples (one triple per line, no abbreviations)
nt_output = g.serialize(format="nt")
print("=== Format N-Triples ===")
print(nt_output)
print()

# Serialize to JSON-LD
jsonld_output = g.serialize(format="json-ld", indent=2)
print("=== Format JSON-LD ===")
print(jsonld_output)

### Interpretation : Comparaison des formats

| Format | Extension | Lisibilite | Usage principal | Equivalent dotNetRDF |
|--------|-----------|-----------|----------------|---------------------|
| **Turtle** | `.ttl` | Excellente | Edition manuelle | `CompressingTurtleWriter` |
| **N-Triples** | `.nt` | Moyenne | Echanges bulk | `NTriplesWriter` |
| **RDF/XML** | `.rdf` | Faible | Compatibilite historique | `RdfXmlWriter` |
| **JSON-LD** | `.jsonld` | Bonne (dev web) | APIs web, Schema.org | `JsonLdWriter` |

> **Conseil** : Preferez Turtle pour le travail quotidien et JSON-LD pour l'integration web.

---

## 5. Charger un Fichier Existant

rdflib peut charger des graphes depuis des fichiers dans differents formats.

In [ ]:
# Load Example.ttl from the data directory
g_example = Graph()
g_example.parse("data/Example.ttl", format="turtle")

print(f"Triples charges depuis Example.ttl : {len(g_example)}")
print()

# Display all triples
print("Contenu de Example.ttl :")
for s, p, o in g_example:
    print(f"  {s.n3(g_example.namespace_manager)} -- {p.n3(g_example.namespace_manager)} --> {o.n3(g_example.namespace_manager)}")

### Interpretation : Chargement de fichier

La methode `g.parse()` charge un fichier RDF dans le graphe.

| Operation | rdflib | dotNetRDF |
|-----------|---------|----------|
| Charger un fichier | `g.parse("file.ttl", format="turtle")` | `FileLoader.Load(g, "file.ttl")` |
| Parser auto-detect | `g.parse("file.ttl")` | `FileLoader.Load(g, "file.ttl")` (auto) |

**Formats supportes par `parse()`** : `turtle`, `xml`, `json-ld`, `nt`, `n3`, `trig`, `nquads`

---

## 6. Tableau de Correspondance dotNetRDF / rdflib

Ce tableau recapitule les equivalences entre les deux bibliotheques pour les operations couvertes dans ce notebook.

| Operation | dotNetRDF (C#) | rdflib (Python) |
|-----------|---------------|----------------|
| **Creer un graphe** | `var g = new Graph();` | `g = Graph()` |
| **Noeud URI** | `g.CreateUriNode(new Uri("..."))` | `URIRef("...")` |
| **Blank node** | `g.CreateBlankNode()` | `BNode()` |
| **Litteral simple** | `g.CreateLiteralNode("text")` | `Literal("text")` |
| **Litteral type** | `g.CreateLiteralNode("42", XSD.Integer)` | `Literal(42, datatype=XSD.integer)` |
| **Litteral avec langue** | `g.CreateLiteralNode("Bonjour", "fr")` | `Literal("Bonjour", lang="fr")` |
| **Ajouter un triple** | `g.Assert(new Triple(s, p, o))` | `g.add((s, p, o))` |
| **Retirer un triple** | `g.Retract(new Triple(s, p, o))` | `g.remove((s, p, o))` |
| **Nombre de triples** | `g.Triples.Count` | `len(g)` |
| **Serialiser en Turtle** | `new CompressingTurtleWriter().Save(g, file)` | `g.serialize(format="turtle")` |
| **Serialiser en N-Triples** | `new NTriplesWriter().Save(g, file)` | `g.serialize(format="nt")` |
| **Charger un fichier** | `FileLoader.Load(g, "file.ttl")` | `g.parse("file.ttl")` |
| **Namespace** | `g.NamespaceMap.AddNamespace("ex", uri)` | `g.bind("ex", Namespace(uri))` |

### Differences philosophiques

| Aspect | dotNetRDF | rdflib |
|--------|-----------|--------|
| **Style** | OOP verbeux, typage fort | Pythonique, concis |
| **Noeuds** | Lies au graphe (`g.CreateUriNode`) | Independants (`URIRef(...)`) |
| **Triples** | Objet `Triple(s, p, o)` | Tuple Python `(s, p, o)` |
| **Iterer** | `foreach (var t in g.Triples)` | `for s, p, o in g:` |

> **A retenir** : Les deux bibliotheques implementent les memes standards W3C. Le choix depend de votre ecosysteme : .NET pour l'integration C#, Python pour la data science et l'IA.

---

## Exercices

Mettez en pratique les concepts appris avec ces deux exercices.

### Exercice 1 : Reproduire le graphe "Hello World" de SW-2

Dans le notebook SW-2 (dotNetRDF), nous avons cree un graphe contenant :
```
<http://www.dotnetrdf.org>  <http://example.org/says>  "Hello World"
<http://www.dotnetrdf.org>  <http://example.org/says>  "Bonjour tout le Monde"@fr
```

Reproduisez ce graphe en Python avec rdflib, puis serialisez-le en Turtle et en N-Triples.

In [ ]:
# Exercice 1 : Reproduire le graphe Hello World de SW-2
# Completez le code ci-dessous

g_hello = Graph()

# Define subject and predicate URIs
dotnetrdf = URIRef("http://www.dotnetrdf.org")
says = URIRef("http://example.org/says")

# TODO: Add the two triples
# g_hello.add((..., ..., ...))  # "Hello World"
# g_hello.add((..., ..., ...))  # "Bonjour tout le Monde"@fr

# Verify
print(f"Nombre de triples : {len(g_hello)} (attendu: 2)")
print()
print("=== Turtle ===")
print(g_hello.serialize(format="turtle"))
print("=== N-Triples ===")
print(g_hello.serialize(format="nt"))

### Exercice 2 : Graphe de livre avec tous les types de noeuds

Crez un graphe RDF decrivant un livre avec :
- Un URI pour le livre (`ex:Book123`)
- Un titre (litteral simple)
- Un nombre de pages (litteral type)
- Un auteur (URI)
- Une description en francais (litteral avec langue)
- Un blank node pour l'editeur (avec nom et URL)

Serialisez le resultat en Turtle.

In [ ]:
# Exercice 2 : Creer un graphe de livre

g_book = Graph()
EX = Namespace("http://example.org/")
g_book.bind("ex", EX)

# TODO: Completez le graphe
# - ex:Book123 a ex:Book
# - ex:Book123 ex:title "..."
# - ex:Book123 ex:pages ... (typed literal)
# - ex:Book123 ex:author ex:Author123
# - ex:Book123 ex:description "..."@fr
# - Create a blank node for publisher with ex:publisherName and ex:publisherURL

print("=== Livre en Turtle ===")
print(g_book.serialize(format="turtle"))

---

## Resume

Ce sidetrack a presente les fondamentaux de **rdflib**, l'equivalent Python de dotNetRDF.

### Concepts cles

| Concept | Ce que vous avez appris |
|---------|------------------------|
| **rdflib** | Creer des graphes, ajouter des triples, gerer les namespaces |
| **Types de noeuds** | URIRef, BNode, Literal (avec datatype et lang) |
| **Serialisation** | Turtle, N-Triples, JSON-LD |
| **Correspondance** | Equivalences dotNetRDF / rdflib |

### Prochaines etapes

- **SW-3-CSharp-GraphOperations** : Manipulation avancee de graphes en .NET (fusion, requetes LINQ)
- **SW-4b-Python-SPARQL** : Le langage de requete SPARQL en Python
- **SW-5b-Python-LinkedData** : Interroger DBpedia et Wikidata avec SPARQLWrapper

---

**Navigation** : [<< SW-2-CSharp-RDFBasics](SW-2-CSharp-RDFBasics.ipynb) | [Index](README.md) | [SW-3-CSharp-GraphOperations >>](SW-3-CSharp-GraphOperations.ipynb)